# ECE 566 Final Project:
### Detecting ArUco Markers and Pose Estimation

Using OpenCV-Python and MATLAB to calibrate the camera and detect ArUco markers off a live camera feed. 

First, we will need to do some setup work.
- Python interpreter (download from internet)
- MATLAB (r2023b preferred)
    - Image processing toolbox
    - Computer vision toolbox
- ipykernel (VS Code should say you need it and give you the option to install if you try to run this script)
- Jupyter Notebooks (VS Code extension)
- pip manager (VS Code extension)
- opencv-contrib-python (pip manager)
- opencv-python (pip manager)
- scipy (pip manager)

Next, run ONLY the first two cells of the notebook. This will fire up a camera.


In [1]:
import cv2
import scipy.io as sio

Now hold up the checkerboard printout you have been given and press 'a' to take a picture. It will tell you when you take a picture. Take 10-11 pictures of the checkerboard at different angles and distances.

In [2]:
c = cv2.VideoCapture(0)

i = 0

while True:
  success, image = c.read()
  cv2.imshow("Image", image)

  # Press 'a' key to save image for calibration
  if cv2.waitKey(33) == ord('a'):
      print("Taking pic {}...".format(i))
      cv2.imwrite("Images/image_{}.png".format(i), image)
      i += 1

  if cv2.waitKey(1) == 27: # ESC key to exit
      break

cv2.destroyAllWindows()

Taking pic 0...
Taking pic 1...
Taking pic 2...
Taking pic 3...
Taking pic 4...
Taking pic 5...
Taking pic 6...
Taking pic 7...
Taking pic 8...
Taking pic 9...
Taking pic 10...


To be able to detect markers and perform pose estimation, we need to calibrate our camera and obtain its parameters. This can be done easily through MATLAB.

Once this is set up, open MATLAB and go to the directory with the unzipped contents. Open the "camera_calibrator_script.m". Make sure the path to the images is correct according to your local machine. Run the script.

MATLAB will then take the images you took and calibrate the camera using that, and save a file called "yourLaptop_camParams.mat". In this file, you will find the camera matrix and distortion coefficients necessary for cv2 to perform pose estimation on the ArUco markers.

Once this is done, you can switch over the notebook again and run the below cells!

In [3]:
camParams = sio.loadmat("yourLaptop_camParams.mat")
cameraMatrix = camParams['cameraMatrix']
distCoeffs = camParams['distortionCoefficients']

In [4]:
camera = cv2.VideoCapture(0)
camera.set(3, 1920)
camera.set(4, 1080)
success = 1

Now that we have our camera initialized to a resolution, and pulled in our calibration coefficients, we need to define an ArUco dictionary and initialize the detector.

We will be working with the 5x5_50 dictionary. This means there are 50 codes and each is 5x5 (25) bits. Basically an ArUco marker is a combination of bits that make a number.

In [10]:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
aruco_params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

Then we set the marker length (actually take a ruler and measure the size of your marker)

In [11]:
markerLength = 142 # mm

Finally, we just have to run our live feed for detection. Below you will find the breakdown of different segments of the code that we will write in.

```
detector.detectMarkers(image)
```
This will take an image and detect all markers. Returns all their corner coordinates, IDs, and anything that it couldn't decipher and rejected as a marker.

```
cv2.aruco.drawDetectedMarkers(image, corners, ids)
```
Draws a box around all the detected markers using their corner coordinates. Then writes their ID in the center of the marker.

```
cv2.aruco.estimatePoseSingleMarkers(markerCorner, markerLength, cameraMatrix, distCoeffs)
```
Take a specific set of corners, the length of the marker, our calibration parameters and gives you the translational and rotational vectors. There are 3 things returned, but we focus on the first 2:
- ```tvec```: The translational vector. Contains the X and Y coordinates on image from the image center pixel, as well as the distance it is from the camera
- ```rvec```: The rotational vector. Contains the angle (in radians) rotation amount in the X, Y, and Z planes

In [12]:
print("Reading from camera...\n")

# To keep track of saved images
i = 0
j = 0

while success:
    success, image = camera.read()
    s = image.shape

    # First we detect all markers in the frame
    (corners, ids, rejected) = detector.detectMarkers(image)

    if len(corners) > 0: # we have detected something
        ids = ids.flatten()
        cv2.aruco.drawDetectedMarkers(image, corners, ids) # draw outlines for all

        # For every detected marker, we do pose estimation using its corners and find the rotational and translational vectors
        for (markerCorner, markerID) in zip(corners, ids):
            reshapedCorners = markerCorner.reshape((4, 2))
            (tL, tR, bR, bL) = reshapedCorners
            topLeft = [int(tL[0]), int(tL[1])]

            rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(markerCorner, markerLength, cameraMatrix, distCoeffs)
            rvec = rvec[0][0]
            tvec = tvec[0][0]

            # Printing distance on the image
            cv2.putText(image, str(round(tvec[2], 2)), (topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print("Marker detected! ID: {}, RVEC: {}, TVEC: {}".format(str(markerID), rvec, tvec))

        # Press 's' key when detecting marker to save image. Only available when marker is detected
        if cv2.waitKey(33) == ord('s'):
            print("Taking ArUco pic {}...".format(j))
            cv2.imwrite("Images/aruco_image_{}.png".format(j), image)
            j += 1

    cv2.imshow("ArUco Detection", image)

    if cv2.waitKey(1) == 27: # ESC key to exit
        break

print("Camera terminated. Finished reading!\n")
cv2.destroyAllWindows()

Reading from camera...



Marker detected! ID: 16, RVEC: [ 2.47222591  0.07263852 -0.9251398 ], TVEC: [275.51210756 211.19922574 749.69202999]
Marker detected! ID: 16, RVEC: [ 2.40776689 -0.00956603 -0.88919938], TVEC: [262.03558012 217.41613541 791.8685961 ]
Marker detected! ID: 16, RVEC: [ 2.39992966 -0.01344138 -0.90870179], TVEC: [261.01745296 219.07900965 794.33620895]
Marker detected! ID: 16, RVEC: [ 3.1300414  -0.0203941   0.16898252], TVEC: [260.49951808 225.27795757 790.9756416 ]
Marker detected! ID: 16, RVEC: [ 3.12436345 -0.02345431  0.18939842], TVEC: [259.57575781 226.13410343 789.02435983]
Marker detected! ID: 16, RVEC: [3.099451   0.0087618  0.26723767], TVEC: [247.32480434 220.04697473 761.4672757 ]
Marker detected! ID: 16, RVEC: [3.10271831e+00 2.84308326e-03 2.69809778e-01], TVEC: [246.33427436 221.61940452 761.91842467]
Marker detected! ID: 16, RVEC: [ 3.10284870e+00 -1.72823137e-03  2.71503986e-01], TVEC: [243.86416585 227.24571377 763.03390606]
Marker detected! ID: 16, RVEC: [ 3.11070367 -0

And there you go! You now know how to use a camera to detect ArUco markers and obtain all vectors from it. You can use this code to make a robot localize to a marker it detects, or choose between different markers using their IDs to localize. The possibilities are endless!

Thanks for taking part! You can continue to learn more about computer vision for more than ArUco marker detection (object detection, depth perception, etc).